# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 8:50AM,258110,12,HH9017,Hush Hush,Released
1,Feb 24 2023 8:49AM,258109,10,SOIVA0000585,Ivaoes Animal Health,Released
2,Feb 24 2023 8:42AM,258106,12,HH-40147,Hush Hush,Released
3,Feb 24 2023 8:42AM,258106,12,HH-40148,Hush Hush,Released
4,Feb 24 2023 8:42AM,258106,12,HH-40149,Hush Hush,Released
5,Feb 24 2023 8:42AM,258106,12,HH-40150,Hush Hush,Released
6,Feb 24 2023 8:42AM,258106,12,HH-40151,Hush Hush,Released
7,Feb 24 2023 8:42AM,258106,12,HH-40152,Hush Hush,Released
8,Feb 24 2023 8:42AM,258106,12,HH-40141,Hush Hush,Released
9,Feb 24 2023 8:42AM,258106,12,HH-40142,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,258105,Released,1
35,258106,Released,28
36,258107,Released,1
37,258109,Released,1
38,258110,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258105,NaN,NaN,1.0
258106,NaN,NaN,28.0
258107,NaN,NaN,1.0
258109,NaN,NaN,1.0
258110,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258031,4.0,2.0,1.0
258039,5.0,14.0,1.0
258041,0.0,0.0,1.0
258044,0.0,1.0,0.0
258052,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258031,4,2,1
258039,5,14,1
258041,0,0,1
258044,0,1,0
258052,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258031,4,2,1
1,258039,5,14,1
2,258041,0,0,1
3,258044,0,1,0
4,258052,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258031,4,2,1
1,258039,5,14,1
2,258041,,,1
3,258044,,1,
4,258052,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 8:50AM,258110,12,Hush Hush
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health
2,Feb 24 2023 8:42AM,258106,12,Hush Hush
30,Feb 24 2023 8:41AM,258107,12,Hush Hush
31,Feb 24 2023 8:38AM,258105,12,Hush Hush
32,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC"
33,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation
37,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc."
42,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation
98,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 8:50AM,258110,12,Hush Hush,,,1
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health,,,1
2,Feb 24 2023 8:42AM,258106,12,Hush Hush,,,28
3,Feb 24 2023 8:41AM,258107,12,Hush Hush,,,1
4,Feb 24 2023 8:38AM,258105,12,Hush Hush,,,1
5,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",,,1
6,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,,,56
7,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",,,5
8,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,,,4
9,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:50AM,258110,12,Hush Hush,1,,
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health,1,,
2,Feb 24 2023 8:42AM,258106,12,Hush Hush,28,,
3,Feb 24 2023 8:41AM,258107,12,Hush Hush,1,,
4,Feb 24 2023 8:38AM,258105,12,Hush Hush,1,,
5,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",1,,
6,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
7,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
8,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
9,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:50AM,258110,12,Hush Hush,1,,
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health,1,,
2,Feb 24 2023 8:42AM,258106,12,Hush Hush,28,,
3,Feb 24 2023 8:41AM,258107,12,Hush Hush,1,,
4,Feb 24 2023 8:38AM,258105,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:50AM,258110,12,Hush Hush,1.0,NaN,NaN
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health,1.0,NaN,NaN
2,Feb 24 2023 8:42AM,258106,12,Hush Hush,28.0,NaN,NaN
3,Feb 24 2023 8:41AM,258107,12,Hush Hush,1.0,NaN,NaN
4,Feb 24 2023 8:38AM,258105,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:50AM,258110,12,Hush Hush,1.0,0.0,0.0
1,Feb 24 2023 8:49AM,258109,10,Ivaoes Animal Health,1.0,0.0,0.0
2,Feb 24 2023 8:42AM,258106,12,Hush Hush,28.0,0.0,0.0
3,Feb 24 2023 8:41AM,258107,12,Hush Hush,1.0,0.0,0.0
4,Feb 24 2023 8:38AM,258105,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4387162,170.0,16.0,9.0
12,1290472,31.0,1.0,0.0
16,774303,3.0,0.0,0.0
18,258104,1.0,0.0,0.0
19,1290447,57.0,3.0,0.0
20,258052,1.0,0.0,0.0
22,258097,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4387162,170.0,16.0,9.0
1,12,1290472,31.0,1.0,0.0
2,16,774303,3.0,0.0,0.0
3,18,258104,1.0,0.0,0.0
4,19,1290447,57.0,3.0,0.0
5,20,258052,1.0,0.0,0.0
6,22,258097,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,170.0,16.0,9.0
1,12,31.0,1.0,0.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,57.0,3.0,0.0
5,20,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,12,Released,31.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,57.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,1.0,0.0,0.0,3.0,0.0,1.0
Released,170.0,31.0,3.0,1.0,57.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,1.0,0.0,0.0,3.0,0.0,1.0
2,Released,170.0,31.0,3.0,1.0,57.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,1.0,0.0,0.0,3.0,0.0,1.0
2,Released,170.0,31.0,3.0,1.0,57.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()